In [1]:
import os
import numpy as np
import numpy.random  as rd
import scipy as sp
from scipy import stats as st
from collections import Counter
import pandas as pd

import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
import matplotlib.animation as ani
import random
import math
plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]
%matplotlib inline

In [2]:
df = pd.read_csv("../data/nikkei225_d.csv")

In [3]:
df.Date = [i.replace("/", "-")for i in df.Date]
df

,Date,Open,High,Low,Close
0,1986-01-04,13118.50,13142.33,13105.97,13136.87
1,1986-01-06,13129.05,13145.54,13053.22,13053.79
2,1986-01-07,13034.57,13037.68,12933.50,12991.24
3,1986-01-08,12997.74,13057.46,12996.80,13056.42
4,1986-01-09,13036.08,13062.26,12959.21,13034.19
5,1986-01-10,13033.91,13037.11,12989.17,12998.21
6,1986-01-13,12992.09,12992.09,12946.87,12977.02
7,1986-01-14,12968.82,12968.82,12924.83,12928.60
8,1986-01-16,12940.18,13027.13,12940.18,13027.13
9,1986-01-17,13031.37,13059.16,12992.28,13009.70


In [4]:
df["Date"] = pd.DatetimeIndex(df["Date"])
df["day_of_week"] = df.Date.dt.dayofweek
df = df.iloc[3540:]
df.iloc[:, 1:-1] = (df.iloc[:, 1:-1]-df.iloc[:, 1:-1].min())/(df.iloc[:, 1:-1].max() - df.iloc[:, 1:-1].min())
df.head()

//anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Date,Open,High,Low,Close,day_of_week
3540,2000-01-04,0.856924,0.872574,0.862694,0.864970,1
3541,2000-01-05,0.861690,0.859283,0.810999,0.831646,2
3542,2000-01-06,0.830704,0.828907,0.807131,0.804550,3
3543,2000-01-07,0.803291,0.807465,0.799895,0.806370,4
3544,2000-01-11,0.807046,0.850913,0.812753,0.853971,1


In [5]:
os.chdir("..")
home_dir = os.getcwd()
os.chdir("src")
from sdle import SDLE
from sdem import SDEM
from smartsifter import *
os.chdir(home_dir)

In [6]:
x = df.day_of_week.reshape((df.shape[0], 1))
y = df.Close.reshape((df.shape[0], 1))
#Z = np.concatenate([x, y], axis=1)
Z = np.concatenate([x, df.iloc[:, 1:-1]], axis=1)

//anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
//anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app


In [7]:
# パラメータ設定
alpha = 2.0
k = 3
T = Z.shape[0]
d = Z.shape[1]-1
r = 1/T * 10
beta = 1
A = range(5)

In [ ]:
# オブジェクトの初期化
sdle = SDLE(r, beta, A)
sdem = [SDEM(r, alpha, k, d) for i in A]

# 同時確率
p = []
# ヘリンジャースコア
S_H = []
# 対数損失
S_L = []

for i, z in enumerate(Z):
    sdle.update(z[0])
    # 各セルに対応する混合ガウス分布を推定
    for m in range(sdle.M):
        if m == int(sdle.flag):
            sdem[m].update(z[1:])
        else:
            sdem[m].skip()
        # tmp: p(y|x)
        tmp = np.dot(sdem[m].prob[sdem[m].t-1],sdem[m].pi[sdem[m].t-1])
    if i == 0:
        p_prev = 0
    else:
        p_prev = p[-1]
    p_t = tmp * sdle.prob[sdle.t-1, int(sdle.flag)]
    p.append(p_t)
    s_h = calc_hellinger_score(sdle.prob[sdle.t-1], sdle.prob[sdle.t-2], sdem, sdle.r, k, sdle.M)
    S_H.append(float(s_h))
    s_l = -np.log(p_prev)
    S_L.append(s_l)

//anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: divide by zero encountered in log


In [ ]:
df["S_L"] = np.array(S_L)
df["S_H"] = np.array(S_H)
df_tmp = df.iloc[1000:].copy()
df.describe()

In [ ]:
# data_uj: ret_n, data_n: ret_uj
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(df.Close)
ax2 = ax.twinx()
ax2.plot(df.S_L,'r')
plt.show()

# data_uj: ret_n, data_n: ret_uj
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(df.Close)
ax2 = ax.twinx()
ax2.plot(df.S_H,'r')
plt.show()

In [ ]:
# data_uj: ret_n, data_n: ret_uj
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(df_tmp.Close)
ax2 = ax.twinx()
ax2.plot(df_tmp.S_L,'r')
plt.show()

# data_uj: ret_n, data_n: ret_uj
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.plot(df_tmp.Close)
ax2 = ax.twinx()
ax2.plot(df_tmp.S_H,'r')
plt.show()